In [40]:
import requests 
import pandas as pd
import time 
import numpy as np

intl_url = 'https://www.signupgenius.com/go/10c0f4aaeab2fa2fac07-july'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(intl_url, headers=header)

In [41]:
dfs = pd.read_html(r.text)
df = dfs[3]
df.rename(columns={0:'Role', 2:'Person'} , inplace = True)
df.drop(columns=1, inplace = True)
df.head(20)

,Role,Person
0,Sunday Greeter (3) (indicate if you have or h...,2 of 3 slots filled Jackie Barber Jac Cottam ...
1,Sunday Secretary,Randy Millar
2,Sunday Topic Bringer,Ricki Gage
3,Monday Greeter (3) (indicate if you have or h...,All slots filled Joshua Reeves Reeves O yeah ...
4,Monday Secretary,JoAnna S
5,Monday Topic Bringer,Brett Hooke
6,Tuesday Greeter (3) (indicate if you have or ...,2 of 3 slots filled Jackie Barber Joshua Reev...
7,Tuesday Secretary,Jackie Barber
8,Tuesday Topic Bringer,JoAnna S
9,Wednesday Greeter (3) (indicate if you have o...,All slots filled Ricki Gage JoAnna S Karen Ca...


# Remove Slots and Signup from Person

In [42]:
to_remove = {
    '2 of 3 slots filled',
    'All slots filled',
    'Sign Up',
    'O yeah'
}

to_replace = {
    'Joshua Reeves Reeves' : 'Joshua Reeves',
    'Bill Collins Collins' : 'Bill Collins'
}

df['Person'] = df['Person'].str.replace('|'.join(to_remove),"")
df['Person'] = df['Person'].str.replace('Joshua Reeves Reeves','Joshua Reeves')
df['Person'] = df['Person'].str.replace('Bill Collins Collins','Bill Collins')

/var/folders/2x/x5mx3_nx79b5zf0xnbz6hnnr0000gn/T/ipykernel_41912/1299371203.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Person'] = df['Person'].str.replace('|'.join(to_remove),"")


# Split Role Column

In [43]:
df['Role'] = df['Role'].str.split('(').str[0]
df.head(50)

verb = []

for i in df['Role']: 
    if 'Greeter' in i:
        i = 'greet(s)!'
    elif 'Topic' in i: 
        i = 'brings topic!'
    elif 'Secretary' in i: 
        i = 'Secretarys!'
    verb.append(i)

df['Verb'] = verb
df.head(40)

,Role,Person,Verb
0,Sunday Greeter,Jackie Barber Jac Cottam,greet(s)!
1,Sunday Secretary,Randy Millar,Secretarys!
2,Sunday Topic Bringer,Ricki Gage,brings topic!
3,Monday Greeter,Joshua Reeves Blake Proctor Karen Cabral,greet(s)!
4,Monday Secretary,JoAnna S,Secretarys!
5,Monday Topic Bringer,Brett Hooke,brings topic!
6,Tuesday Greeter,Jackie Barber Joshua Reeves,greet(s)!
7,Tuesday Secretary,Jackie Barber,Secretarys!
8,Tuesday Topic Bringer,JoAnna S,brings topic!
9,Wednesday Greeter,Ricki Gage JoAnna S Karen Cabral,greet(s)!


In [44]:
df['First Person'] = df['Person'].str.split().str[0]
df['Second Person'] = df['Person'].str.split().str[2]
df['Third Person'] = df['Person'].str.split().str[4]
df = df.fillna('')
df.head(50)

,Role,Person,Verb,First Person,Second Person,Third Person
0,Sunday Greeter,Jackie Barber Jac Cottam,greet(s)!,Jackie,Jac,
1,Sunday Secretary,Randy Millar,Secretarys!,Randy,,
2,Sunday Topic Bringer,Ricki Gage,brings topic!,Ricki,,
3,Monday Greeter,Joshua Reeves Blake Proctor Karen Cabral,greet(s)!,Joshua,Blake,Karen
4,Monday Secretary,JoAnna S,Secretarys!,JoAnna,,
5,Monday Topic Bringer,Brett Hooke,brings topic!,Brett,,
6,Tuesday Greeter,Jackie Barber Joshua Reeves,greet(s)!,Jackie,Joshua,
7,Tuesday Secretary,Jackie Barber,Secretarys!,Jackie,,
8,Tuesday Topic Bringer,JoAnna S,brings topic!,JoAnna,,
9,Wednesday Greeter,Ricki Gage JoAnna S Karen Cabral,greet(s)!,Ricki,JoAnna,Karen


In [47]:
df.to_csv(r'data.csv')